# Project: Finding Lines On the Road
- **Environment: python 3.6**

## Ideas of lane detection pipeline
 - cv2.inRange() for color selection
 - cv2.fillPoly() for regions selection
 - cv2.line() to draw lines on an image
 - cv2.addWeighted() to coadd/overlay two images
 - cv2.imwrite() to output images to file
 - cv2.itwise_and() to apply a mask to an image

## Import packages

In [655]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math

## Read image and convert to gray scale

In [656]:
def readImage(path):
    image = cv2.imread(path)
    return image

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def drawImage(img):
    plt.imshow(img)

## Find edge

In [657]:
def gaussian_blur(img, kernal_size):
    return cv2.GaussianBlur(img, (kernal_size, kernal_size), 0)
    
def findEdge(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

## Find region of interests

In [658]:
def regionOfInterest(img, vertices):
    mask = np.zeros_like(img)
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

## Hough lines
 - lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
 - rho in units of pixels and theta in units of radians, rho takes a minimum value of 1, and a reasonable starting place for theta is 1 degree (pi/180 in radians).
 - the threshold parameter specifies the minimum number of votes (intersections in a given grid cell) 
 - the empty np.array([]) is just a placeholder, no need to change it.
 - min_line_length is the minimum length of a line (in pixels) that you will accept in the output, 
 - max_line_gap is the maximum distance (again, in pixels) between segments that you will allow to be connected into a single line. 


In [659]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    # Returns an image with hough lines drawn.
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
#     line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
#     draw_lines(line_img, lines)
    return lines

## Find best line
 - First, find the main lines. 
 - Next, find the groups of lines that are similar to eachother (by comparing slope and bias), and save these as "the same line." 
 - Next, take the two most common lines, and assume these must be our lanes. After we've done ROI, the next most likely "line" just simply is almost certain to be the lanes. That's the hypothesis anyway!

**ms (slop) value is very important for finding lines**

In [660]:
from numpy import ones,vstack
from numpy.linalg import lstsq
from statistics import mean

def findBestLines(shape, lines):
    try:
        ys = []  
        for i in lines:
            for ii in i:
                ys += [ii[1],ii[3]]
        min_y = min(ys)
        max_y = shape[0]
        new_lines = []
        line_dict = {}
        for idx,i in enumerate(lines):
            for xyxy in i:
                x_coords = (xyxy[0],xyxy[2])
                y_coords = (xyxy[1],xyxy[3])
                A = vstack([x_coords,ones(len(x_coords))]).T
                m, b = lstsq(A, y_coords)[0]

                # Calculating our new, and improved, xs
                x1 = (min_y-b) / m
                x2 = (max_y-b) / m

                line_dict[idx] = [m,b,[int(x1), min_y, int(x2), max_y]]
        final_lanes = {}

        for idx in line_dict:
            final_lanes_copy = final_lanes.copy()
            m = line_dict[idx][0]
            b = line_dict[idx][1]
            line = line_dict[idx][2]
            
            if len(final_lanes) == 0:
                final_lanes[m] = [ [m,b,line] ]
                
            else:
                found_copy = False

                for other_ms in final_lanes_copy:

                    if not found_copy:
                        leftLineSlop = 1.1
                        rightLineSlop = 0.93
                        if abs(other_ms*leftLineSlop) > abs(m) > abs(other_ms*rightLineSlop):
                            if abs(final_lanes_copy[other_ms][0][1]*leftLineSlop) > abs(b) > abs(final_lanes_copy[other_ms][0][1]*rightLineSlop):
                                final_lanes[other_ms].append([m,b,line])
                                found_copy = True
                                break
                        else:
                            final_lanes[m] = [ [m,b,line] ]
        line_counter = {}

        for lanes in final_lanes:
            line_counter[lanes] = len(final_lanes[lanes])
        top_lanes = sorted(line_counter.items(), key=lambda item: item[1])[::-1][:2]
#         print('debug',top_lanes,len(top_lanes))
        if(len(top_lanes)==2):
            lane1_id = top_lanes[0][0]
            lane2_id = top_lanes[1][0]
        else:
            lane1_id = top_lanes[0][0]
            lane2_id = top_lanes[0][0]

        def average_lane(lane_data):
            x1s = []
            y1s = []
            x2s = []
            y2s = []
            for data in lane_data:
                x1s.append(data[2][0])
                y1s.append(data[2][1])
                x2s.append(data[2][2])
                y2s.append(data[2][3])
            return int(mean(x1s)), int(mean(y1s)), int(mean(x2s)), int(mean(y2s)) 

        l1_x1, l1_y1, l1_x2, l1_y2 = average_lane(final_lanes[lane1_id])
        l2_x1, l2_y1, l2_x2, l2_y2 = average_lane(final_lanes[lane2_id])

        return [l1_x1, l1_y1, l1_x2, l1_y2], [l2_x1, l2_y1, l2_x2, l2_y2]
    
    except Exception as e:
        print(str(e))

## Draw lines on image

In [661]:
def draw_lines(shape, lines, color=[0, 0, 255], thickness=3):
    line_img = np.zeros((shape[0], shape[1], 3), dtype=np.uint8)
    try:
        for line in lines:
            cv2.line(line_img, (line[0], line[1]), (line[2], line[3]), color, thickness)
        return line_img
    except Exception as e:
        print('draw lines error:',e)

In [662]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    try:
        return cv2.addWeighted(initial_img, α, img, β, γ)
    except Exception as e:
        print('weighted_image err:',e)

## Save images

In [663]:
def saveImg(img, path):
    try:
        cv2.imwrite(path, img)
    except Exception as e:
        print('same image error',e)

## Test images
 - Build your pipeline that will draw lane lines on the test_images
 - Save them to the test_images_output directory.

In [664]:
import os
from tqdm import tqdm 

DIR = 'test_images'
for name in tqdm(os.listdir(DIR)):
    path = os.path.join(DIR, name)
    image = readImage(path)
    gray = grayscale(image)
    edge = findEdge(gaussian_blur(gray,3), 150, 300)
    # create vertices and roi image
    left_bottom = (0, image.shape[0])
    right_bottom = (image.shape[1], image.shape[0])
    left_top = (image.shape[1]/3, image.shape[0]*2/3)
    right_top = (image.shape[1]*2/3, image.shape[0]*2/3)
    vertices = np.array([[left_top, right_top, right_bottom, left_bottom]], dtype=np.int32)
    maskedImage = regionOfInterest(edge, vertices)
    
    # create hough lines
    rho = 1
    theta = np.pi/180
    threshold = 1
    min_line_length = 20
    max_line_gap = 15
    lines = hough_lines(maskedImage, rho, theta, 
                        threshold, min_line_length, max_line_gap)
    print('========')
    bestLines = findBestLines(image.shape, lines)
    print('bestlines',bestLines)
    line_img = draw_lines(image.shape, bestLines)
    # draw on image
    result = weighted_img(line_img, image)
    saveImg(result, f'test_image_output/{name}.jpg')
#     plt.imshow(result)
#     break

UsageError: Cell magic `%%cache` not found.


## Test on videos
 - To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
 - To do so add .subclip(start_second,end_second) to the end of the line below
 - Where start_second and end_second are integer values representing the start and end of the subclip
 - You may also uncomment the following line for a subclip of the first 5 seconds
 - clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)

In [ ]:
def process_image(img, white_output):
    pass

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
# import os

file_path = 'test_videos/solidWhiteRight.mp4'
white_output = 'test_videos_output/solidWhiteRight.mp4'

clip1 = VideoFileClip(file_path)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
white_clip.write_videofile(white_output, audio=False)      